In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import joblib
import os
import pandas as pd
import gzip

In [2]:
def transform_label(num):
    if num == 0:
        return 'negative'
    elif num == 1:
        return 'neutral'
    elif num == 2:
        return 'positive'
    else:
        raise ValueError("Invalid label")

In [3]:
train = pd.read_csv('data/processed_train.csv')
tfidf_vec = TfidfVectorizer(max_features=10000).fit(train['text'])
bog_vec = CountVectorizer(max_features=10000).fit(train['text'])
binary_vec = CountVectorizer(max_features=10000, binary=True).fit(train['text'])

In [5]:
k_means_model = joblib.load('weights/k_means.pkl')
random_forest_model = joblib.load('weights/random_forest.pkl')
logistic_model = joblib.load('weights/logistic_regression.pkl')
with gzip.open("weights/knn.pkl.gz", "rb") as f:
    knn_model = joblib.load(f)
with gzip.open("weights/svm.pkl.gz", "rb") as f:
    svm_model = joblib.load(f)

In [6]:
def inference(model, text):
    if model == 'knn':
        x_predict = tfidf_vec.transform([text]).toarray()
        sentiment = transform_label(knn_model.predict(x_predict))
    elif model == 'random_forest':
        x_predict = bog_vec.transform([text]).toarray()
        sentiment = transform_label(random_forest_model.predict(x_predict))
    elif model == 'logistic_regression':
        x_predict = bog_vec.transform([text]).toarray()
        sentiment = transform_label(logistic_model.predict(x_predict))
    elif model == 'svm':
        x_predict = tfidf_vec.transform([text]).toarray()
        sentiment = transform_label(svm_model.predict(x_predict))
    elif model == 'kmeans':
        x_predict = tfidf_vec.transform([text]).toarray()
        sentiment = transform_label(k_means_model.predict(x_predict))
    return sentiment

In [9]:
print(inference('svm', 'I love this product!'))

positive
